<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/MissForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/WideTable/WideTable.xlsx')
df = df.iloc[:, 1:]
df.head()

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,L14:WBC,Sex,Age,NATNL,Location,PricipalProblemCode,PricipalProblemName,InPatient,OutPatient,Emergency
0,2299,5812902,-7.158596,100,100,34,0,0,0,0,...,9.41,หญิง,75.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0.0,1.0,0.0
1,2299,6947220,5.550691,35,0,1,35,0,0,0,...,NaN,หญิง,76.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0.0,1.0,0.0
2,2299,8186767,-6.063710,100,0,15,0,0,0,0,...,5.42,หญิง,78.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0.0,1.0,0.0
3,2299,9385133,-12.812245,90,0,0,0,0,0,0,...,NaN,หญิง,79.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,F411,Generalized anxiety disorder,0.0,1.0,0.0
4,2304,4916314,28.287500,0,0,0,0,0,0,0,...,NaN,หญิง,58.0,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,NaN,NaN,0.0,1.0,0.0


In [3]:
lab = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Lab.csv')

In [4]:
import pandas as pd

# Step 1: Compute mode of 'Location' for each 'PatientUID' in 'lab' table
mode_location = lab.groupby('PatientUID')['Location'].apply(lambda x: x.mode()[0]).reset_index()

# Step 2: Create a dictionary mapping 'PatientUID' to mode 'Location'
mode_dict = dict(zip(mode_location['PatientUID'], mode_location['Location']))

# Step 3: Fill null values in the 'Location' column of 'df' with the mode values
df['Location'] = df.apply(lambda row: mode_dict[row['PatientUID']] if pd.isnull(row['Location']) else row['Location'], axis=1)


In [5]:
data = pd.DataFrame(df)
remove = ['L12:Hemoglobin','L13:Hematocrit','L16:TIBC','L17:Tranferin SAT',
          'L18:Ferritin','L25:Parathyroid hormone','L26:Vitamin D','L27:UrineAlbumin/Cr ratio',
          'L28:UrineProtein/Cr ratio','L29:UrineVolume (24hrs)','L30:UrineUrea (24hrs)',
          'L31:UrineSodium (24hrs)','L32:UrineProtein (24hrs)']
data = data.drop(remove, axis=1)

In [6]:
data['Age'] = data['Age'].fillna('69')
data['Sex'] = data['Sex'].fillna('หญิง')
data['NATNL'] = data['NATNL'].fillna('ไทย')
data['InPatient'] = data['InPatient'].fillna(0)
data['OutPatient'] = data['OutPatient'].fillna(1)
data['Emergency'] = data['Emergency'].fillna(0)
data['PricipalProblemCode'] = data['PricipalProblemCode'].fillna('None')
data['PricipalProblemName'] = data['PricipalProblemName'].fillna('None')

## **MissForest All**

In [9]:
import pandas as pd
column_to_exclude = 'Slopes'
null_columns = data.columns[data.isnull().any()]
numeric_non_null_columns = data.select_dtypes(include=[float, int]).columns[~data.select_dtypes(include=[float, int]).isnull().any() & (data.select_dtypes(include=[float, int]).columns != column_to_exclude)]
data[null_columns] = data[null_columns].apply(pd.to_numeric, errors='coerce')
correlation_matrix = data.corr()
correlation = correlation_matrix.loc[null_columns, numeric_non_null_columns]

<ipython-input-9-cb709e0075f0>:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = data.corr()


In [10]:
print(data[null_columns].dtypes)

Systolic BP            float64
Diastolic BP           float64
BMI                    float64
BSA                    float64
L2:BUN                 float64
L3:Creatinine          float64
L4:Sodium              float64
L5:Potassium           float64
L6:Chloride            float64
L7:Bicarbonate         float64
L8:Uric                float64
L9:Calcium             float64
L10:Phosphorus         float64
L11:Albumin            float64
L15:Platelet           float64
L19:Blood sugar        float64
L20:HbA1c              float64
L21:Cholesterol        float64
L22:Triglyceride       float64
L23:LDL Cholesterol    float64
L24:HDL Cholesterol    float64
L14:WBC                float64
dtype: object


In [11]:
!pip install missingpy
!pip install MissForest
import pandas as pd
import numpy as np
from missforest.missforest import MissForest

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 1.4 MB/s eta 0:00:00


In [12]:
# Store the original column names
original_columns = {}
for col in null_columns:
    original_columns[col.replace(" ", "").replace(":", "").replace("-", "").replace("/", "").replace("&", "").replace("#", "")] = col

In [13]:
# Rename only null_missforest columns
for col in null_columns:
    data = data.rename(columns={col: col.replace(" ", "").replace(":", "").replace("-", "").replace("/", "").replace("&", "").replace("#", "")})


In [14]:
# Columns to impute using MissForest #22
columns_to_impute = ['SystolicBP', 'DiastolicBP', 'BMI', 'BSA', 'L5Potassium',
                     'L7Bicarbonate', 'L8Uric', 'L9Calcium', 'L10Phosphorus',
                     'L11Albumin', 'L14WBC', 'L15Platelet', 'L19Bloodsugar',
                     'L20HbA1c', 'L22Triglyceride', 'L24HDLCholesterol',
                     'L2BUN','L3Creatinine','L4Sodium','L6Chloride',
                     'L21Cholesterol','L23LDLCholesterol']
data[columns_to_impute]

,SystolicBP,DiastolicBP,BMI,BSA,L5Potassium,L7Bicarbonate,L8Uric,L9Calcium,L10Phosphorus,L11Albumin,...,L19Bloodsugar,L20HbA1c,L22Triglyceride,L24HDLCholesterol,L2BUN,L3Creatinine,L4Sodium,L6Chloride,L21Cholesterol,L23LDLCholesterol
0,138.0,76.0,20.72,1.38,3.8,29.7,NaN,NaN,NaN,NaN,...,100.0,5.3,62.0,65.0,6.5,0.82,143.0,103.0,193.0,129.0
1,134.0,66.0,20.27,1.36,4.3,NaN,NaN,NaN,NaN,NaN,...,89.0,NaN,69.0,76.0,NaN,0.88,NaN,NaN,205.0,139.0
2,103.0,57.0,19.05,1.33,4.4,27.2,NaN,NaN,NaN,NaN,...,88.0,5.5,74.0,60.0,14.5,1.10,141.0,102.0,198.0,129.0
3,128.0,62.0,NaN,NaN,4.2,28.2,NaN,NaN,NaN,NaN,...,93.0,5.3,71.0,68.0,15.4,0.72,131.0,96.0,179.0,100.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,111.0,6.2,97.0,51.0,20.8,1.08,NaN,NaN,143.0,89.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,147.0,87.0,NaN,NaN,4.6,27.4,NaN,NaN,NaN,NaN,...,135.0,6.6,146.0,48.0,NaN,2.04,137.0,99.0,181.0,98.0
52615,143.0,86.0,NaN,NaN,5.2,23.5,NaN,9.2,4.0,4.2,...,133.0,7.2,104.0,47.0,28.3,1.87,141.0,104.0,125.0,62.0
52616,131.0,79.0,NaN,NaN,4.4,19.4,10.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,15.5,1.09,141.0,110.0,NaN,NaN
52617,138.0,NaN,NaN,NaN,4.5,22.6,7.3,NaN,NaN,3.2,...,NaN,NaN,NaN,NaN,40.8,2.14,126.0,90.0,121.0,NaN


In [15]:

# Initialize the MissForest imputer
imputer = MissForest()

# Fit and transform the specified columns using MissForest
data_imputed = imputer.fit_transform(data[columns_to_impute])


/usr/local/lib/python3.10/dist-packages/missforest/missforest.py:240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[c].fillna(impute_vals, inplace=True)
/usr/local/lib/python3.10/dist-packages/missforest/missforest.py:240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[c].fillna(impute_vals, inplace=True)
/usr/local/lib/python3.10/dist-packages/missforest/missforest.py:240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[c].fillna(i

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000202 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1453
[LightGBM] [Info] Number of data points in the train set: 395, number of used features: 21
[LightGBM] [Info] Start training from score 136.131646
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

/usr/local/lib/python3.10/dist-packages/missforest/missforest.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_imp.loc[miss_index, c] = y_pred
/usr/local/lib/python3.10/dist-packages/missforest/missforest.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_imp.loc[miss_index, c] = y_pred


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/usr/local/lib/python3.10/dist-packages/missforest/missforest.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_imp.loc[miss_index, c] = y_pred


...

/usr/local/lib/python3.10/dist-packages/missforest/missforest.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_imp.loc[miss_index, c] = y_pred


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000113 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1434
[LightGBM] [Info] Number of data points in the train set: 395, number of used features: 21
[LightGBM] [Info] Start training from score 171.164557
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

/usr/local/lib/python3.10/dist-packages/missforest/missforest.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_imp.loc[miss_index, c] = y_pred


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000115 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1440
[LightGBM] [Info] Number of data points in the train set: 395, number of used features: 21
[LightGBM] [Info] Start training from score 101.886076
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

/usr/local/lib/python3.10/dist-packages/missforest/missforest.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_imp.loc[miss_index, c] = y_pred


In [16]:
data[columns_to_impute]

,SystolicBP,DiastolicBP,BMI,BSA,L5Potassium,L7Bicarbonate,L8Uric,L9Calcium,L10Phosphorus,L11Albumin,...,L19Bloodsugar,L20HbA1c,L22Triglyceride,L24HDLCholesterol,L2BUN,L3Creatinine,L4Sodium,L6Chloride,L21Cholesterol,L23LDLCholesterol
0,138.0,76.0,20.72,1.38,3.8,29.7,NaN,NaN,NaN,NaN,...,100.0,5.3,62.0,65.0,6.5,0.82,143.0,103.0,193.0,129.0
1,134.0,66.0,20.27,1.36,4.3,NaN,NaN,NaN,NaN,NaN,...,89.0,NaN,69.0,76.0,NaN,0.88,NaN,NaN,205.0,139.0
2,103.0,57.0,19.05,1.33,4.4,27.2,NaN,NaN,NaN,NaN,...,88.0,5.5,74.0,60.0,14.5,1.10,141.0,102.0,198.0,129.0
3,128.0,62.0,NaN,NaN,4.2,28.2,NaN,NaN,NaN,NaN,...,93.0,5.3,71.0,68.0,15.4,0.72,131.0,96.0,179.0,100.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,111.0,6.2,97.0,51.0,20.8,1.08,NaN,NaN,143.0,89.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,147.0,87.0,NaN,NaN,4.6,27.4,NaN,NaN,NaN,NaN,...,135.0,6.6,146.0,48.0,NaN,2.04,137.0,99.0,181.0,98.0
52615,143.0,86.0,NaN,NaN,5.2,23.5,NaN,9.2,4.0,4.2,...,133.0,7.2,104.0,47.0,28.3,1.87,141.0,104.0,125.0,62.0
52616,131.0,79.0,NaN,NaN,4.4,19.4,10.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,15.5,1.09,141.0,110.0,NaN,NaN
52617,138.0,NaN,NaN,NaN,4.5,22.6,7.3,NaN,NaN,3.2,...,NaN,NaN,NaN,NaN,40.8,2.14,126.0,90.0,121.0,NaN


In [17]:
data_imputed

,SystolicBP,DiastolicBP,BMI,BSA,L5Potassium,L7Bicarbonate,L8Uric,L9Calcium,L10Phosphorus,L11Albumin,...,L19Bloodsugar,L20HbA1c,L22Triglyceride,L24HDLCholesterol,L2BUN,L3Creatinine,L4Sodium,L6Chloride,L21Cholesterol,L23LDLCholesterol
0,138.000000,76.000000,20.720000,1.380000,3.800000,29.700000,4.956932,9.698268,3.344167,4.479870,...,100.000000,5.300000,62.000000,65.000000,6.500000,0.82,143.000000,103.000000,193.000000,129.000000
1,134.000000,66.000000,20.270000,1.360000,4.300000,27.416059,5.329132,9.663190,3.618002,4.378406,...,89.000000,5.758289,69.000000,76.000000,13.827432,0.88,143.882932,106.355843,205.000000,139.000000
2,103.000000,57.000000,19.050000,1.330000,4.400000,27.200000,5.807531,9.238514,3.908119,3.990624,...,88.000000,5.500000,74.000000,60.000000,14.500000,1.10,141.000000,102.000000,198.000000,129.000000
3,128.000000,62.000000,25.984196,1.723299,4.200000,28.200000,5.016248,9.512573,3.488127,4.524066,...,93.000000,5.300000,71.000000,68.000000,15.400000,0.72,131.000000,96.000000,179.000000,100.000000
4,136.380522,66.373506,24.459945,1.755030,4.401381,26.454919,6.531964,9.365163,3.721122,4.603736,...,111.000000,6.200000,97.000000,51.000000,20.800000,1.08,140.844669,103.283883,143.000000,89.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,147.000000,87.000000,28.586896,1.870644,4.600000,27.400000,7.218941,9.826285,3.500366,4.706140,...,135.000000,6.600000,146.000000,48.000000,22.522226,2.04,137.000000,99.000000,181.000000,98.000000
52615,143.000000,86.000000,24.544091,1.593430,5.200000,23.500000,6.807546,9.200000,4.000000,4.200000,...,133.000000,7.200000,104.000000,47.000000,28.300000,1.87,141.000000,104.000000,125.000000,62.000000
52616,131.000000,79.000000,21.592746,1.651981,4.400000,19.400000,10.900000,9.607657,3.352559,4.458130,...,123.908794,6.225853,125.093974,45.598665,15.500000,1.09,141.000000,110.000000,140.827638,85.063666
52617,138.000000,64.648452,24.915457,1.451846,4.500000,22.600000,7.300000,8.371686,3.890922,3.200000,...,157.323269,7.056111,99.202594,40.219694,40.800000,2.14,126.000000,90.000000,121.000000,62.149598


In [18]:
data[columns_to_impute] = data_imputed

In [19]:
# Rename the columns back to their original state
data = data.rename(columns=original_columns)

In [20]:
data

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,L14:WBC,Sex,Age,NATNL,Location,PricipalProblemCode,PricipalProblemName,InPatient,OutPatient,Emergency
0,2299,5812902,-7.158596,100,100,34,0,0,0,0,...,9.410000,หญิง,75.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0.0,1.0,0.0
1,2299,6947220,5.550691,35,0,1,35,0,0,0,...,5.070692,หญิง,76.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0.0,1.0,0.0
2,2299,8186767,-6.063710,100,0,15,0,0,0,0,...,5.420000,หญิง,78.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0.0,1.0,0.0
3,2299,9385133,-12.812245,90,0,0,0,0,0,0,...,4.930649,หญิง,79.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,F411,Generalized anxiety disorder,0.0,1.0,0.0
4,2304,4916314,28.287500,0,0,0,0,0,0,0,...,6.564400,หญิง,58.0,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,0,0,0,0,0,0,0,...,5.508229,ชาย,64.0,ไทย,หน่วยคลินิกพิเศษศูนย์บริการทางการแพทย์ชั้นเลิศ,44054006,Type 2 diabetes mellitus,0.0,1.0,0.0
52615,3319971,10553916,-4.256753,0,0,0,0,0,0,0,...,7.239943,หญิง,72.0,ไทย,หน่วยคลินิกพิเศษศูนย์บริการทางการแพทย์ชั้นเลิศ,44054006,Type 2 diabetes mellitus,0.0,1.0,0.0
52616,3320048,10572126,-0.692078,0,0,0,0,0,0,0,...,8.972605,ชาย,67.0,ไทย,TB (คลินิก)-D707,A150,"Tuberculosis of lung\, confirmed by sputum mic...",0.0,1.0,0.0
52617,3320050,10572565,7.217875,0,0,0,0,0,0,0,...,18.700000,ชาย,68.0,ไทย,โรคเลือดและต่อมน้ำเหลือง (คลินิก)-D716,414166008,"Extranodal NK/T-cell lymphoma, nasal type",0.0,1.0,0.0


In [21]:
null_count = data.isnull().sum()
null_columns = null_count[null_count > 0]
non_null_columns = null_count[null_count == 0]
print(null_columns)

Series([], dtype: int64)


In [22]:
data.to_excel('MissForest.xlsx')